# Networkx And Global Flow of Funds

In [20]:
import csv
import networkx as nx 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
plt.style.use('ggplot')
%matplotlib inline

In [21]:
## read pre-processed data from stata
df = pd.read_stata('data/0_CPIS_CDIS_BIS_USTIC_merged_fixed.dta')

In [22]:
## print the variables that we have in our data 
df.columns.values

array(['country', 'countrycode', 'counterpart', 'counterpart_code', 'year',
       'CDIS_IADE', 'CDIS_IADD', 'CDIS_IAD', 'CDIS_IADF', 'CPIS_IAP',
       'CPIS_IAPE', 'CPIS_IAPD', 'loans_dep'], dtype=object)

In [27]:
keep_var = ['countrycode','counterpart_code','country','counterpart','CDIS_IAD','CPIS_IAP','loans_dep']
df_2015 = df[df['year']==2015][keep_var]
df_2015.fillna(0,inplace=True)
df_2015.loc[df['CDIS_IAD']<0,'CDIS_IAD'] = 0                       # recode all negative values to 0 
df_2015.loc[df['CPIS_IAP']<0,'CPIS_IAP'] = 0                       # recode negative values to 0 
## df_2015['log_CDIS_IAD'] = np.log(df_2015['CDIS_IAD'])        # Take log 

In [28]:
df_2015.head()

,countrycode,counterpart_code,country,counterpart,CDIS_IAD,CPIS_IAP,loans_dep
7,316.0,111.0,Barbados,United States,3285.000000,4575.566589,0.0
21,960.0,111.0,Croatia,United States,2.619745,2084.000000,0.0
22,443.0,111.0,Kuwait,United States,741.560120,2064.070740,0.0
29,514.0,111.0,Bhutan,United States,0.000000,50.000000,0.0
30,722.0,111.0,Senegal,United States,0.000000,4.000000,0.0


In [29]:
df_2015.describe()

,countrycode,counterpart_code,CDIS_IAD,CPIS_IAP,loans_dep
count,31745.000000,31745.000000,3.174500e+04,3.174500e+04,31745.000000
mean,522.174133,525.499756,1.205572e+03,1.323649e+03,421.135837
std,287.403625,286.209991,1.790810e+04,2.159184e+04,9055.887223
min,111.000000,111.000000,0.000000e+00,0.000000e+00,0.000000
25%,238.000000,248.000000,0.000000e+00,0.000000e+00,0.000000
50%,532.000000,534.000000,0.000000e+00,0.000000e+00,0.000000
75%,746.000000,748.000000,1.113362e-01,0.000000e+00,0.000000
max,968.000000,968.000000,1.115748e+06,1.369423e+06,940793.000000


In [30]:
## see how mancy countries we have
print(df_2015['country'].unique().shape)
print(df_2015['counterpart'].unique().shape)

(233,)
(235,)


### Now, we dump data into a network 

In [61]:
# Create a network with pandas dataframe 
df_2015 = df_2015[df_2015.CDIS_IAD>0]
df_2015['rev_CDIS_IAD'] = 1/df_2015.CDIS_IAD
G = nx.from_pandas_dataframe(df_2015, source="country", target="counterpart", edge_attr=['CDIS_IAD','rev_CDIS_IAD'],create_using=nx.DiGraph()) ##,'CPIS_IAP','loans_dep'

In [62]:
# print out one particular edge, just to make sure it wroks 
G['United States']['United Kingdom']

{'CDIS_IAD': 717895.0, 'rev_CDIS_IAD': 1.3929613942309516e-06}

In [63]:
# make sure the graph is created correctly 
df_2015[(df_2015.country == 'United States') & (df_2015.counterpart == 'United Kingdom')]

,countrycode,counterpart_code,country,counterpart,CDIS_IAD,CPIS_IAP,loans_dep,rev_CDIS_IAD
2811,111.0,112.0,United States,United Kingdom,717895.0,1244554.0,614951.0,0.000001


In [64]:
print(nx.info(G))

Name: 
Type: DiGraph
Number of nodes: 234
Number of edges: 8872
Average in degree:  37.9145
Average out degree:  37.9145


In [65]:
print('Degree of US: {}'.format((G.degree("United States"))))
print('Degree of UK: {}'.format((G.degree("United Kingdom"))))

Degree of US: 193
Degree of UK: 249


### First look at degree centrality

In [66]:
### calculate some centrality mearuses 
### This is the sum of both in and out folows 
d=G.degree(weight='CDIS_IAD')                                 ## get the degree centrality for all countries 
cdis_degree = pd.DataFrame(list(d.items()),columns=['country','CDIS_IAD'])      ## make it into dataframe
cdis_degree.sort_values(by='CDIS_IAD',ascending=0,inplace=True)                 ## sort it 
cdis_degree.head(10)

,country,CDIS_IAD
149,United States,1.077993e+07
103,Netherlands,9.037645e+06
216,Luxembourg,7.958590e+06
189,United Kingdom,5.546120e+06
153,Germany,3.294401e+06
92,"China, P.R.: Hong Kong",3.163979e+06
128,France,2.506216e+06
56,"China, P.R.: Mainland",2.492785e+06
97,Switzerland,2.451718e+06
220,Ireland,2.448264e+06



### Closeness Centrality

In [90]:
## for closeness centrality, there is a problem here, 
## we simply coded weigt = 0 for countries with no connection, but this may have an impact on btwettness, closeness,etc calculation
#nx.shortest_path_length(G)['United States']
c=nx.closeness_centrality(G)#distance ='rev_CDIS_IAD'                                  ## use CDIS_IAD data as distance mearuse, so the order should be reversed
cdis_closeness = pd.DataFrame(list(c.items()),columns=['country','CDIS_IAD'])      ## make it into dataframe
cdis_closeness.sort_values(by='CDIS_IAD',ascending=0,inplace=True)    
cdis_closeness.head(10)

,country,CDIS_IAD
196,Italy,0.934764
153,Germany,0.838891
191,"Korea, Republic of",0.797969
10,Brazil,0.738765
213,Denmark,0.724738
128,France,0.720180
135,Sub-Saharan Africa,0.715679
55,Belgium,0.713449
6,Mauritius,0.700358
22,Poland,0.698223


## Betweeness

In [96]:
b = nx.betweenness_centrality(G)
cdis_between = pd.DataFrame(list(b.items()),columns=['country','CDIS_IAD'])      ## make it into dataframe
cdis_between.sort_values(by='CDIS_IAD',ascending=0,inplace=True)    
cdis_between.head(10)

,country,CDIS_IAD
196,Italy,0.192156
153,Germany,0.081278
191,"Korea, Republic of",0.045279
135,Sub-Saharan Africa,0.044780
10,Brazil,0.039908
79,Thailand,0.038741
56,"China, P.R.: Mainland",0.028956
23,Russian Federation,0.028655
6,Mauritius,0.027918
213,Denmark,0.024680


## Eigenvector Centrality

In [101]:
e = nx.eigenvector_centrality_numpy(G,weight='CDIS_IAD')
cdis_eigenvector = pd.DataFrame(list(e.items()),columns=['country','CDIS_IAD'])      ## make it into dataframe
cdis_eigenvector.sort_values(by='CDIS_IAD',ascending=0,inplace=True)    
cdis_eigenvector.head(10)

,country,CDIS_IAD
149,United States,0.494055
103,Netherlands,0.453902
189,United Kingdom,0.423208
216,Luxembourg,0.383761
220,Ireland,0.217005
97,Switzerland,0.169624
153,Germany,0.165636
31,Canada,0.134037
232,Bermuda,0.131964
55,Belgium,0.107828


## Density measure

In [106]:
## density of network 
den = nx.density(G)
den

0.16272330435420565

### Diameter

In [109]:
## diameter of net work 
try:
    nx.diameter(G)
except:
    print("Some nodes is not reachable, network is weekly connected")

Some nodes is not reachable, network is weekly connected


### Convert graph to adj matrix 

In [358]:
### create adj matrix from a graph 

## sort node list order 
node_list = G.nodes()
node_list.sort()
adj_matrix = nx.to_numpy_matrix(G = G,nodelist=node_list,weight ='CDIS_IAD')
adj_matrix